In [1]:
import sqlite3
import cv2
import numpy as np
import os
def cap_save(db_path, cascade_path):
    
    try:
        # Establish connection to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        # Load the Haar cascade classifier
        faceCascade = cv2.CascadeClassifier(cascade_path)
        # Start the webcam
        cap = cv2.VideoCapture(0)

        while True:
            
            ret, frame = cap.read()

            # Check if the frame is received correctly?
            if not ret:
                print("Error: Cannot receive frame from webcam")
                break
                
            # Detect faces using the cascade classifier
            faces = faceCascade.detectMultiScale(
                frame,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(150, 150),
                flags=cv2.CASCADE_SCALE_IMAGE
            )

            # For each face in the frame:
            for (x, y, w, h) in faces:
                cv2.putText(frame, 'please click c', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
                roi = frame[y:y+h, x:x+w]

                # Convert to grayscale 
               # gray_image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

            
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            cv2.imshow('Webcam', frame)

            key = cv2.waitKey(1)
            if key == ord('c'):       
                cv2.imwrite(filename='saved_img.jpg',img = roi )
                cap.release()
                img_new = cv2.imread('saved_img.jpg')
                denoised_roi = cv2.fastNlMeansDenoising(img_new)
                img_new = cv2.imshow("Captured Image", denoised_roi)
                cv2.waitKey(10000)
                cv2.destroyAllWindows()
                break

        # Convert the denoised ROI to byte array for database storage
        image_bytes = cv2.imencode('.jpg', denoised_roi)[1].tobytes()

        # Get a unique image ID (assuming an auto-incrementing ID in the table)
        cursor.execute("SELECT MAX(image_id) FROM images")
        max_id = cursor.fetchone()[0]
        if max_id is None:
            image_id = 1
        else:
            image_id = max_id + 1

        # Prompt user for name
        name = input("Enter name for the captured face: ")
        
        
        ''' Our table in the database has three columns named image_id,image_name and image_data . '''
        

        # Insert the image data and name into the database
        sql = "INSERT INTO images (image_id, image_name, image_data) VALUES (?, ?, ?)"
        cursor.execute(sql, (image_id, name, image_bytes))
        conn.commit()
        print("Face captured and saved successfully!")
    except (cv2.Error, sqlite3.Error) as e:
          print(f"An error occurred: {e}")
    
    finally:
        # Close resources regardless of exceptions
        if cap.isOpened():
            cv2.waitKey(0)
            cap.release()
        cv2.destroyAllWindows()
        conn.close()
    return len(faces) > 0  # Return True if a face was captured
while True:
    #db_path = input("Enter the path to the SQLite database file (e.g., C:/path/to/faces.db): ")
    db_path = 'C:\\Users\\asus\\Desktop\\test\\faces2.db'
    if not os.path.exists(db_path):
        print(f"Error: Database file '{db_path}' does not exist. Please create it.")
    cascade_path = "C:\\Users\\asus\\Desktop\\test\\haarcascade_frontalface_default.xml"
    #cascade_path = input("Enter the path to the Haar cascade classifier: ")
    if not os.path.exists(cascade_path):
        print(f"Error: cascade file '{cascade_path}' does not exist.")
    
    success = cap_save(db_path, cascade_path)

    if success:
        repeat = input("Do you want to capture another face? (y/n): ")
        if repeat.lower() != 'y':
            break
    else:
        repeat = input("An error occurred. Do you want to try capturing another face? (y/n): ")
        if repeat.lower() != 'y':
            break








Enter name for the captured face: gf
Face captured and saved successfully!
Do you want to capture another face? (y/n): n
